In [ ]:
# Parameters
kedro_env = "test_cloud"


## Imports

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import yaml

## Loads config files

In [ ]:
with open('../conf/base/parameters.yml') as f:
    # use safe_load instead load
    parameters = yaml.safe_load(f)

with open('../conf/base/catalog.yml') as f:
    # use safe_load instead load
    catalog = yaml.safe_load(f)    

## Query using kedro catalog

In [ ]:
from kedro_datasets.pandas import GBQQueryDataSet

bq_query_cat_entry = catalog['bq_test']
sql = bq_query_cat_entry['sql']
project_id = bq_query_cat_entry['project']

data_set = GBQQueryDataSet(sql, project=project_id)
sql_data = data_set.load()

## Query using bigquery API (useful for debugging of queries)

In [ ]:
# loads BigQuery project parameters
bq_query_cat_entry = catalog['bq_test']
project_id = bq_query_cat_entry['project']

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)


query = """ SELECT name, SUM(number) as total_people
    FROM bigquery-public-data.usa_names.usa_1910_2013
    WHERE state = 'TX'
    GROUP BY name, state
    ORDER BY total_people DESC
    LIMIT 20 """
query_job = client.query(query)  # Make an API request.

print("The query data:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print("name={}, count={}".format(row[0], row["total_people"]))